In [1]:
%%writefile matmul.cu
#include <cuda_runtime.h>
#include <stdio.h>

// Kernel 定义
__global__ void matrix_multiplication_kernel(const float* A, const float* B, float* C, int M, int N, int K) {
    int row =  blockDim.y * blockIdx.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    
    if(col >= K || row >= M){
        return;
    }
    
    float acc = 0.0f;
    for(int i = 0; i < N; i++){
        acc += A[row * N + i] * B[i * K + col];
    }
    C[row * K + col] = acc; 
}

// 宿主端 wrapper 函数
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 threadsPerBlock(16, 16);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrix_multiplication_kernel<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);
    
    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }
    
    // 等待 GPU 完成
    cudaDeviceSynchronize();
}

Overwriting matmul.cu


上面的实现有一个很大的问题就是，在每一个线程里面，为了计算C里面的一个位置上面的值，需要去全局内存上访问col次数据A，也需要访问col次数据B，导致数据等待时间大大增加。解决方法在于，使用线程块里面的共享内存，在一个线程块里面，先读取A和B的一部分数据到线程块里面的共享内存里面，然后进行计算，接着读取下一部分。通过分块避免所有线程去读取全局内存里面的数据。

In [2]:
%%writefile matmul_v2.cu
#include <stdio.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 32 // 假设 BlockDim 为 32x32

__global__ void matrix_multiplication_shared_mem(const float* __restrict__ A, const float* __restrict__ B, float* C, int M, int N, int K) {
    // 申请共享内存
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

    int bx = blockIdx.x;
    int by = blockIdx.y;
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // 计算当前线程在全局矩阵 C 中负责的坐标
    int row = by * TILE_WIDTH + ty;
    int col = bx * TILE_WIDTH + tx;

    float acc = 0.0f;

    // 循环遍历所有的 Tile (以 TILE_WIDTH 为步长遍历 N 维度)
    for (int t = 0; t < (N + TILE_WIDTH - 1) / TILE_WIDTH; ++t) {

        // 1. 协作加载 A 的 Tile 到共享内存
        // 边界检查：防止索引越界 (Padding 0)
        if (row < M && t * TILE_WIDTH + tx < N) {
            As[ty][tx] = A[row * N + t * TILE_WIDTH + tx];
        } else {
            As[ty][tx] = 0.0f;
        }

        // 2. 协作加载 B 的 Tile 到共享内存
        if (col < K && t * TILE_WIDTH + ty < N) {
            // 注意这里 B 的索引：行是 t*TILE_WIDTH + ty, 列是 col
            Bs[ty][tx] = B[(t * TILE_WIDTH + ty) * K + col];
        } else {
            Bs[ty][tx] = 0.0f;
        }

        // 3. 必须同步！确保所有线程都加载完了数据
        __syncthreads();

        // 4. 在共享内存上进行计算
        for (int i = 0; i < TILE_WIDTH; ++i) {
            acc += As[ty][i] * Bs[i][tx];
        }

        // 5. 必须同步！确保在加载下一个 Tile 之前，当前 Tile 的数据已经用完
        __syncthreads();
    }

    // 写回结果
    if (row < M && col < K) {
        C[row * K + col] = acc;
    }
}

// 宿主端 wrapper 函数
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);
    
    matrix_multiplication_shared_mem<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);
    
    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }
    
    // 等待 GPU 完成
    cudaDeviceSynchronize();
}

Overwriting matmul_v2.cu


In [3]:
%%writefile matmul_v3.cu
#include <stdio.h>
#include <cuda_runtime.h>
#define TILE_WIDTH 32 // 假设 BlockDim 为 32x32

__global__ void matrix_multiplication_shared_mem(const float *__restrict__ A, const float *__restrict__ B, float *C, int M, int N, int K)
{
    // 申请共享内存
    __shared__ float As[TILE_WIDTH][TILE_WIDTH];
    __shared__ float Bs[TILE_WIDTH][TILE_WIDTH+1];

    int ty = threadIdx.y;
    int tx = threadIdx.x;

    int row = blockIdx.y * TILE_WIDTH + ty;
    int col = blockIdx.x * TILE_WIDTH + tx;

    float acc = 0.0f;

    for (int i = 0; i < (N + TILE_WIDTH - 1) / TILE_WIDTH; i++)
    {
        // 读取数据
        if (row < M && TILE_WIDTH * i + tx < N)
        {
            As[ty][tx] = A[row * N + TILE_WIDTH * i + tx];
        }
        else
        {
            As[ty][tx] = 0.0f;
        }
        if (col < K && TILE_WIDTH * i + ty < N)
        {
            Bs[ty][tx] = B[(TILE_WIDTH * i + ty) * K + col];
        }
        else
        {
            Bs[ty][tx] = 0.0f;
        }
        __syncthreads(); // 等待线程块里面的线程都搬运完成

        for (int j = 0; j < TILE_WIDTH; j++)
        {
            acc += As[ty][j] * Bs[j][tx]; //访问线程块里面的共享内存时，没有合并访问，只在乎bank config
        }

        __syncthreads();
    }
    if (row<M && col<K)
    {
        C[row*K+col] = acc;
    }
}

// 宿主端 wrapper 函数
extern "C" void solve(const float *A, const float *B, float *C, int M, int N, int K)
{
    dim3 threadsPerBlock(TILE_WIDTH, TILE_WIDTH);
    // 注意：grid 的计算需要向上取整，你的代码已经包含了这个逻辑，但建议加上括号保证运算顺序
    dim3 blocksPerGrid((K + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (M + threadsPerBlock.y - 1) / threadsPerBlock.y);

    matrix_multiplication_shared_mem<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, M, N, K);

    // 检查是否有错误发生（调试用）
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess)
    {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }

    // 等待 GPU 完成
    cudaDeviceSynchronize();
}


Overwriting matmul_v3.cu


In [4]:
%%writefile matmul_v4.cu
#include <stdio.h>
#include <cuda_runtime.h>

// 宏定义块大小
// TS (Tile Size): 每个 Block 计算 64x64 的 C
// WPT (Work Per Thread): 每个线程计算 4x4 的 C
// TS_K: K 维度(你的代码里是 N 维度)的分块大小，设为 8 或 16
#define TS 64
#define WPT 4
#define TS_K 16 

// 优化后的 Kernel
__global__ void matrix_multiplication_optimized(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    // 每个 Block 处理 C 中 TS x TS (64x64) 的区域
    // 线程块维度: dim3(TS/WPT, TS/WPT) -> (16, 16) -> 256 个线程
    
    // 1. 声明共享内存
    // As: 存储 A 的切片 [TS][TS_K] -> [64][16]
    // Bs: 存储 B 的切片 [TS_K][TS] -> [16][64]
    __shared__ float As[TS][TS_K];
    __shared__ float Bs[TS_K][TS];

    // 2. 声明寄存器
    // accum: 累加器，每个线程负责计算 4x4 = 16 个元素
    float accum[WPT][WPT] = {0.0f};
    
    // reg_A, reg_B: 用于在内循环中缓存从 SMEM 读取的值
    float reg_A[WPT];
    float reg_B[WPT];

    // 线程 ID 和 Block ID
    int tx = threadIdx.x; // range 0-15
    int ty = threadIdx.y; // range 0-15
    int bx = blockIdx.x;
    int by = blockIdx.y;

    // 当前线程负责的 C 矩阵起始坐标 (C 的分块左上角 + 线程偏移)
    // 每个线程覆盖 WPT(4) 个像素宽/高
    int row_c = by * TS + ty * WPT; 
    int col_c = bx * TS + tx * WPT;

    // 3. 循环遍历 N 维度 (步长 TS_K = 16)
    for (int t = 0; t < N; t += TS_K) {
        
        // --- 加载数据到 Shared Memory (协作加载) ---
        // 我们有 256 个线程。
        // 需要加载 A 的 Tile: 64行 * 16列 = 1024 元素。每个线程加载 1024/256 = 4 个元素。
        // 需要加载 B 的 Tile: 16行 * 64列 = 1024 元素。每个线程加载 4 个元素。
        
        // 加载 As (A 的子块): 
        // 这里的逻辑是将 256 个线程映射到 64x16 的区域
        // 我们使用 float4 向量化加载来极致优化带宽
        
        // 计算当前线程加载 As 的位置
        // 将 16x16 的线程块视为 256 个线性线程
        int tid = ty * (TS / WPT) + tx; // 0 ~ 255
        
        // 映射到 As[64][16]: 每一行 16 个元素，如果是 float4 就是 4 个 float4
        // 256 个线程，每个加载 1 个 float4 (4个float)，正好 1024 个 float
        // As 的行索引
        int load_a_row = tid / (TS_K / 4); 
        int load_a_col = (tid % (TS_K / 4)) * 4;
        
        // 从全局内存 A 加载到 As
        // 全局索引: A[(by * TS + load_a_row) * N + (t + load_a_col)]
        // 注意边界检查省略了，假设维度对其
        if (by * TS + load_a_row < M && t + load_a_col < N) {
             // 使用 float4 指针强转进行向量加载
             float4 tmp = reinterpret_cast<const float4*>(&A[(by * TS + load_a_row) * N + (t + load_a_col)])[0];
             As[load_a_row][load_a_col + 0] = tmp.x;
             As[load_a_row][load_a_col + 1] = tmp.y;
             As[load_a_row][load_a_col + 2] = tmp.z;
             As[load_a_row][load_a_col + 3] = tmp.w;
        }

        // 加载 Bs (B 的子块): [16][64]
        // 同样用 tid 映射。每行 64 个元素 = 16 个 float4。
        // 总共 16 行。总 float4 数 = 16 * 16 = 256。正好每个线程取 1 个 float4。
        int load_b_row = tid / (TS / 4);
        int load_b_col = (tid % (TS / 4)) * 4;

        if (t + load_b_row < N && bx * TS + load_b_col < K) {
             float4 tmp = reinterpret_cast<const float4*>(&B[(t + load_b_row) * K + (bx * TS + load_b_col)])[0];
             Bs[load_b_row][load_b_col + 0] = tmp.x;
             Bs[load_b_row][load_b_col + 1] = tmp.y;
             Bs[load_b_row][load_b_col + 2] = tmp.z;
             Bs[load_b_row][load_b_col + 3] = tmp.w;
        }

        __syncthreads(); // 等待数据加载完成

        // --- 在寄存器上进行计算 ---
        // 遍历 Shared Memory 中的 TS_K (16) 维度
        #pragma unroll
        for (int k = 0; k < TS_K; ++k) {
            
            // 1. 将所需的 As 和 Bs 数据预加载到寄存器
            // 每个线程计算 4x4，需要 As 的一列 4 个值，Bs 的一行 4 个值
            for (int i = 0; i < WPT; ++i) {
                reg_A[i] = As[ty * WPT + i][k];
                reg_B[i] = Bs[k][tx * WPT + i];
            }

            // 2. 外积计算 (Outer Product)
            // 计算 4x4 的结果，复用 reg_A 和 reg_B
            for (int row = 0; row < WPT; ++row) {
                for (int col = 0; col < WPT; ++col) {
                    accum[row][col] += reg_A[row] * reg_B[col];
                }
            }
        }
        
        __syncthreads(); // 等待计算完成，准备加载下一块
    }

    // 4. 写回结果到全局内存
    // 每个线程写回 4x4 个点
    for (int row = 0; row < WPT; ++row) {
        for (int col = 0; col < WPT; ++col) {
            int global_row = row_c + row;
            int global_col = col_c + col;
            
            if (global_row < M && global_col < K) {
                C[global_row * K + global_col] = accum[row][col];
            }
        }
    }
}

// Host 端调用示例
extern "C" void solve(const float* d_A, const float* d_B, float* d_C, int M, int N, int K) {
    // 线程块大小: 16x16 = 256 线程
    dim3 threadsPerBlock(TS / WPT, TS / WPT); 
    
    // Grid 大小: 因为每个 Block 处理 64x64，所以除以 TS(64)
    dim3 numBlocks((K + TS - 1) / TS, (M + TS - 1) / TS);

    matrix_multiplication_optimized<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, M, N, K);
}

Overwriting matmul_v4.cu


In [5]:
%%writefile matmul_v4_1.cu
#include <stdio.h>
#include <cuda_runtime.h>

// 宏定义块大小
// TS (Tile Size): 每个 Block 计算 64x64 的 C
// WPT (Work Per Thread): 每个线程计算 4x4 的 C
// TS_K: K 维度(你的代码里是 N 维度)的分块大小，设为 8 或 16
#define TS 64
#define WPT 4
#define TS_K 16 

// 优化后的 Kernel
__global__ void matrix_multiplication_optimized(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    // 每个 Block 处理 C 中 TS x TS (64x64) 的区域
    // 线程块维度: dim3(TS/WPT, TS/WPT) -> (16, 16) -> 256 个线程
    
    // 1. 声明共享内存
    // As: 存储 A 的切片 [TS][TS_K] -> [64][16]
    // Bs: 存储 B 的切片 [TS_K][TS] -> [16][64]
    __shared__ float As[TS][TS_K];
    __shared__ float Bs[TS_K][TS];

    // 2. 声明寄存器
    // accum: 累加器，每个线程负责计算 4x4 = 16 个元素
    float accum[WPT][WPT] = {0.0f};
    
    // reg_A, reg_B: 用于在内循环中缓存从 SMEM 读取的值
    float reg_A[WPT];
    float reg_B[WPT];

    // 线程 ID 和 Block ID
    int tx = threadIdx.x; // range 0-15
    int ty = threadIdx.y; // range 0-15
    int bx = blockIdx.x;
    int by = blockIdx.y;

    // 当前线程负责的 C 矩阵起始坐标 (C 的分块左上角 + 线程偏移)
    // 每个线程覆盖 WPT(4) 个像素宽/高
    int row_c = by * TS + ty * WPT; 
    int col_c = bx * TS + tx * WPT;

    // 3. 循环遍历 N 维度 (步长 TS_K = 16)
    for (int t = 0; t < N; t += TS_K) {
        
        // --- 加载数据到 Shared Memory (协作加载) ---
        // 我们有 256 个线程。
        // 需要加载 A 的 Tile: 64行 * 16列 = 1024 元素。每个线程加载 1024/256 = 4 个元素。
        // 需要加载 B 的 Tile: 16行 * 64列 = 1024 元素。每个线程加载 4 个元素。
        
        // 加载 As (A 的子块): 
        // 这里的逻辑是将 256 个线程映射到 64x16 的区域
        // 我们使用 float4 向量化加载来极致优化带宽
        
        // 计算当前线程加载 As 的位置
        // 将 16x16 的线程块视为 256 个线性线程
        int tid = ty * (TS / WPT) + tx; // 0 ~ 255
        
        // 映射到 As[64][16]: 每一行 16 个元素，如果是 float4 就是 4 个 float4
        // 256 个线程，每个加载 1 个 float4 (4个float)，正好 1024 个 float
        // As 的行索引
        int load_a_row = tid / (TS_K / 4); 
        int load_a_col = (tid % (TS_K / 4)) * 4;
        
        // 从全局内存 A 加载到 As
        // 全局索引: A[(by * TS + load_a_row) * N + (t + load_a_col)]
        // 注意边界检查省略了，假设维度对其
        if (by * TS + load_a_row < M && t + load_a_col < N) {
             // 使用 float4 指针强转进行向量加载
             float4 tmp = reinterpret_cast<const float4*>(&A[(by * TS + load_a_row) * N + (t + load_a_col)])[0];
             As[load_a_row][load_a_col + 0] = tmp.x;
             As[load_a_row][load_a_col + 1] = tmp.y;
             As[load_a_row][load_a_col + 2] = tmp.z;
             As[load_a_row][load_a_col + 3] = tmp.w;
        }

        // 加载 Bs (B 的子块): [16][64]
        // 同样用 tid 映射。每行 64 个元素 = 16 个 float4。
        // 总共 16 行。总 float4 数 = 16 * 16 = 256。正好每个线程取 1 个 float4。
        int load_b_row = tid / (TS / 4);
        int load_b_col = (tid % (TS / 4)) * 4;

        if (t + load_b_row < N && bx * TS + load_b_col < K) {
             float4 tmp = reinterpret_cast<const float4*>(&B[(t + load_b_row) * K + (bx * TS + load_b_col)])[0];
             Bs[load_b_row][load_b_col + 0] = tmp.x;
             Bs[load_b_row][load_b_col + 1] = tmp.y;
             Bs[load_b_row][load_b_col + 2] = tmp.z;
             Bs[load_b_row][load_b_col + 3] = tmp.w;
        }

        __syncthreads(); // 等待数据加载完成

        // --- 在寄存器上进行计算 ---
        // 遍历 Shared Memory 中的 TS_K (16) 维度
        #pragma unroll
        for (int k = 0; k < TS_K; ++k) {
            
            // 1. 将所需的 As 和 Bs 数据预加载到寄存器
            // 每个线程计算 4x4，需要 As 的一列 4 个值，Bs 的一行 4 个值
            for (int i = 0; i < WPT; ++i) {
                reg_A[i] = As[ty * WPT + i][k];
                reg_B[i] = Bs[k][tx * WPT + i];
            }

            // 2. 外积计算 (Outer Product)
            // 计算 4x4 的结果，复用 reg_A 和 reg_B
            for (int row = 0; row < WPT; ++row) {
                for (int col = 0; col < WPT; ++col) {
                    accum[row][col] += reg_A[row] * reg_B[col];
                }
            }
        }
        
        __syncthreads(); // 等待计算完成，准备加载下一块
    }

    // 4. 写回结果到全局内存
    // 每个线程写回 4x4 个点
    for (int row = 0; row < WPT; ++row) {
        for (int col = 0; col < WPT; ++col) {
            int global_row = row_c + row;
            int global_col = col_c + col;
            
            if (global_row < M && global_col < K) {
                C[global_row * K + global_col] = accum[row][col];
            }
        }
    }
}

// Host 端调用示例
extern "C" void solve(const float* d_A, const float* d_B, float* d_C, int M, int N, int K) {
    // 线程块大小: 16x16 = 256 线程
    dim3 threadsPerBlock(TS / WPT, TS / WPT); 
    
    // Grid 大小: 因为每个 Block 处理 64x64，所以除以 TS(64)
    dim3 numBlocks((K + TS - 1) / TS, (M + TS - 1) / TS);

    matrix_multiplication_optimized<<<numBlocks, threadsPerBlock>>>(d_A, d_B, d_C, M, N, K);
}

Overwriting matmul_v4_1.cu


In [1]:
%%writefile matmul_v5.cu
#include <stdio.h>
#include <cuda_runtime.h>

// ==========================================
// 针对 T4 (Compute Capability 7.5) 优化的参数
// ==========================================
// 每个 Block 计算 128x128 的 C
const int BM = 128;
const int BN = 128;
// K 维度的步进，每次加载 8 列
const int BK = 8;
// 每个线程计算 8x8 的 C
const int TM = 8;
const int TN = 8;

// 线程块大小: (BM/TM) * (BN/TN) = 16 * 16 = 256 线程
// 满足 T4 最佳 Occupancy

__global__ __launch_bounds__(256)
void sgemm_optimized_t4(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    // 线程索引
    int tid = threadIdx.x; // 0..255
    
    // 逻辑坐标 (16x16)
    int ty = tid / 16;
    int tx = tid % 16;

    // 当前 Block 负责的 C 的左上角坐标
    int by = blockIdx.y;
    int bx = blockIdx.x;
    
    // Shared Memory 声明
    // As: 存储 A 的切片 [BK][BM]。
    // 注意：这里我们将 A 转置存储 (Transposed)，为了后续计算时能向量化读取
    // Padding: 为了避免 Bank Conflicts，我们在行尾 +1 或 +4 (这里不需要，因为 128 是 32 的倍数，但读取方式不同)
    // 实际上，为了极致性能，我们让 As 的维度为 [BK][BM]，这样计算时按 As[k][row] 读取是连续的。
    __shared__ float As[BK][BM]; 
    __shared__ float Bs[BK][BN];

    // 寄存器累加器，8x8
    float accum[TM][TN] = {0.0f};

    // 用于从 SMEM 加载到寄存器的临时变量
    float rag[TM]; // 缓存 A 的一列
    float rbg[TN]; // 缓存 B 的一行

    // 计算当前 Block 在 Global Memory 中的起始位置
    const float* A_ptr = A + by * BM * K;
    const float* B_ptr = B + bx * BN;
    float* C_ptr = C + by * BM * N + bx * BN;

    // ==========================================================
    // 预计算加载 Global Memory 的索引
    // 我们有 256 个线程。
    // A 的 Tile 是 128(Row) x 8(Col)。总共 1024 元素。每个线程搬运 4 个 (float4)。
    // B 的 Tile 是 8(Row) x 128(Col)。总共 1024 元素。每个线程搬运 4 个 (float4)。
    // ==========================================================

    // A 的加载索引 (为了转置存储到 SMEM)
    // 我们按 Global A 的行优先读取，写入到 As 的 [col][row]
    // tid 范围 0-255。
    // 128行 * 8列 / 4(float4) = 256 个 float4 操作。正好 1 线程 1 个 float4。
    // load_a_row: 0..127, load_a_col: 0, 4 (因为 K step 是 8)
    int load_a_row = tid / 2; 
    int load_a_col = (tid % 2) * 4;

    // B 的加载索引
    // 8行 * 128列 / 4 = 256 个 float4。
    int load_b_row = tid / 32;
    int load_b_col = (tid % 32) * 4;

    // 主循环：在 K 维度上推进
    for (int k_step = 0; k_step < K; k_step += BK) {
        
        // --------------------------------------------------------
        // 1. 加载数据到 Shared Memory (Vectorized Global Load)
        // --------------------------------------------------------
        
        // 加载 A (M x K): 使用 float4
        // 边界检查：假设 M, N, K 是 8/128 的倍数以获得最佳性能，这里加简单防护
        if (by * BM + load_a_row < M && k_step + load_a_col < K) {
            float4 tmp = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + k_step + load_a_col])[0];
            // 关键优化：转置写入 As
            // Global: A[row][k] -> SMEM: As[k][row]
            // 这样在计算阶段，同一个 k 的不同 row 是连续内存
            As[load_a_col + 0][load_a_row] = tmp.x;
            As[load_a_col + 1][load_a_row] = tmp.y;
            As[load_a_col + 2][load_a_row] = tmp.z;
            As[load_a_col + 3][load_a_row] = tmp.w;
        } else {
            // 边界 padding (设为0不影响累加)
             As[load_a_col + 0][load_a_row] = 0.0f;
             As[load_a_col + 1][load_a_row] = 0.0f;
             As[load_a_col + 2][load_a_row] = 0.0f;
             As[load_a_col + 3][load_a_row] = 0.0f;
        }

        // 加载 B (K x N): 使用 float4
        if (k_step + load_b_row < K && bx * BN + load_b_col < N) {
            float4 tmp = reinterpret_cast<const float4*>(&B_ptr[(k_step + load_b_row) * N + load_b_col])[0];
            // B 不需要转置，直接按行存
            reinterpret_cast<float4*>(&Bs[load_b_row][load_b_col])[0] = tmp;
        } else {
             Bs[load_b_row][load_b_col + 0] = 0.0f;
             Bs[load_b_row][load_b_col + 1] = 0.0f;
             Bs[load_b_row][load_b_col + 2] = 0.0f;
             Bs[load_b_row][load_b_col + 3] = 0.0f;
        }

        __syncthreads();

        // --------------------------------------------------------
        // 2. 计算 (Math Loop)
        // --------------------------------------------------------
        // 展开循环，计算 8 个 K 步骤
        #pragma unroll
        for (int k = 0; k < BK; ++k) {
            // 从 SMEM 加载 A 的一列 (TM=8) 到寄存器
            // 由于我们转置了 As，现在 As[k][row...] 是连续的！
            // 我们可以用 float4 加载来加速！
            // 当前线程计算的 C 的行是: ty * TM 到 ty * TM + 7
            // 对应的 SMEM 地址是: &As[k][ty * TM]
            float4 tmpA0 = reinterpret_cast<const float4*>(&As[k][ty * TM])[0];
            float4 tmpA1 = reinterpret_cast<const float4*>(&As[k][ty * TM + 4])[0];
            
            rag[0] = tmpA0.x; rag[1] = tmpA0.y; rag[2] = tmpA0.z; rag[3] = tmpA0.w;
            rag[4] = tmpA1.x; rag[5] = tmpA1.y; rag[6] = tmpA1.z; rag[7] = tmpA1.w;

            // 从 SMEM 加载 B 的一行 (TN=8) 到寄存器
            // 当前线程计算的 C 的列是: tx * TN 到 tx * TN + 7
            // 对应的 SMEM 地址是: &Bs[k][tx * TN]
            float4 tmpB0 = reinterpret_cast<const float4*>(&Bs[k][tx * TN])[0];
            float4 tmpB1 = reinterpret_cast<const float4*>(&Bs[k][tx * TN + 4])[0];

            rbg[0] = tmpB0.x; rbg[1] = tmpB0.y; rbg[2] = tmpB0.z; rbg[3] = tmpB0.w;
            rbg[4] = tmpB1.x; rbg[5] = tmpB1.y; rbg[6] = tmpB1.z; rbg[7] = tmpB1.w;

            // 外积 (Outer Product) 计算 8x8
            // 编译器会自动优化为 FFMA 指令
            #pragma unroll
            for (int r = 0; r < TM; ++r) {
                #pragma unroll
                for (int c = 0; c < TN; ++c) {
                    accum[r][c] += rag[r] * rbg[c];
                }
            }
        }

        __syncthreads();
    }

    // --------------------------------------------------------
    // 3. 写回 Global Memory
    // --------------------------------------------------------
    // 每个线程负责 8x8 个点。
    // 为了带宽优化，我们也应该用 float4 写回。
    // 这稍微复杂一点，因为 thread 的 8x8 是块状的，不是完全连续的行。
    // 每个线程有 8 行，每行 8 个元素。每行的 8 个元素是连续的。
    // 可以用 2 个 float4 写回一行。

    int global_row_start = by * BM + ty * TM;
    int global_col_start = bx * BN + tx * TN;

    #pragma unroll
    for (int r = 0; r < TM; ++r) {
        int global_r = global_row_start + r;
        if (global_r < M) {
            int global_c = global_col_start;
            if (global_c + 7 < N) {
                // 常见的路径：可以直接 float4 写回
                float4 tmp0;
                tmp0.x = accum[r][0]; tmp0.y = accum[r][1]; tmp0.z = accum[r][2]; tmp0.w = accum[r][3];
                reinterpret_cast<float4*>(&C[global_r * N + global_c])[0] = tmp0;

                float4 tmp1;
                tmp1.x = accum[r][4]; tmp1.y = accum[r][5]; tmp1.z = accum[r][6]; tmp1.w = accum[r][7];
                reinterpret_cast<float4*>(&C[global_r * N + global_c + 4])[0] = tmp1;
            } else {
                // 边界处理
                for (int c = 0; c < TN; ++c) {
                    if (global_c + c < N) {
                        C[global_r * N + global_c + c] = accum[r][c];
                    }
                }
            }
        }
    }
}

// 主机端调用 Wrapper
extern "C" void solve(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 block(256);
    dim3 grid((N + BN - 1) / BN, (M + BM - 1) / BM);
    
    sgemm_optimized_t4<<<grid, block>>>(A, B, C, M, N, K);
}

// 测试 main 函数 (可选)
int main() {
    int M = 4096;
    int N = 4096;
    int K = 4096;
    size_t size = M * K * sizeof(float); // 简化测试，假设方阵

    float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;
    
    // 分配内存... (略去分配和初始化代码以保持简洁)
    // cudaMalloc(&d_A, ...);
    
    // 调用
    // launch_sgemm_optimized(d_A, d_B, d_C, M, N, K);
    
    // cudaDeviceSynchronize();
    
    printf("Kernel compiled and structure ready for T4.\n");
    return 0;
}

Writing matmul_v5.cu


In [5]:
import os
import subprocess
for v in os.listdir(os.path.abspath('.')):
    prefix, end = os.path.splitext(v)
    if end == '.cu':
        subprocess.run(f"nvcc -shared -o {prefix}.so {prefix}.cu -Xcompiler -fPIC", shell=True)
print(os.listdir())

matmul_v5.cu(55): warning #177-D: variable "C_ptr" was declared but never referenced
      float* C_ptr = C + by * BM * N + bx * BN;
             ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

matmul_v5.cu(206): warning #177-D: variable "N" was declared but never referenced
      int N = 4096;
          ^

matmul_v5.cu(208): warning #177-D: variable "size" was declared but never referenced
      size_t size = M * K * sizeof(float);
             ^

matmul_v5.cu(210): warning #177-D: variable "h_A" was declared but never referenced
      float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;
             ^

matmul_v5.cu(210): warning #177-D: variable "h_B" was declared but never referenced
      float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;
                   ^

matmul_v5.cu(210): warning #177-D: variable "h_C" was declared but never referenced
      float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;
                         ^

matmul_v5.cu(210): warning #177-D: variable "d_A" w

['matmul_v5.cu', '.virtual_documents', 'matmul_v5.so', 'matmul_v6.cu']


matmul_v6.cu(208): error: identifier "BN" is undefined
      dim3 grid((N + BN - 1) / BN, (M + BM - 1) / BM);
                     ^

matmul_v6.cu(208): error: identifier "BM" is undefined
      dim3 grid((N + BN - 1) / BN, (M + BM - 1) / BM);
                                        ^

2 errors detected in the compilation of "matmul_v6.cu".


In [3]:
import torch
import ctypes
import numpy as np
import time

# --- 测试部分 ---

# 设置维度
M, N, K = 1024, 1024, 1024

print(f"正在进行矩阵乘法测试: [{M}x{N}] * [{N}x{K}]")

# 创建随机数据 (在 GPU 上)
device = torch.device("cuda")
A = torch.randn(M, N, device=device, dtype=torch.float32)
B = torch.randn(N, K, device=device, dtype=torch.float32)

# 2. 运行 PyTorch 内置矩阵乘法 (作为标准答案)
C_torch = torch.matmul(A, B)

print("Pytorch: ")
%timeit torch.matmul(A, B); torch.cuda.synchronize()
print()

for v in sorted(os.listdir('.')):
    prefix, end = os.path.splitext(v)
    
    if end != '.so':
        continue

    # 1. 加载编译好的 .so 库
    lib = ctypes.CDLL(f'./{v}')

    # 2. 定义函数参数类型
    # void solve(const float* A, const float* B, float* C, int M, int N, int K)
    # 指针对应 c_void_p (因为我们要传显存地址), int 对应 c_int
    lib.solve.argtypes = [
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_void_p, 
        ctypes.c_int, 
        ctypes.c_int, 
        ctypes.c_int
    ]

    def cuda_matmul(a_tensor, b_tensor):
        # 获取维度
        M, N = a_tensor.shape
        N_b, K = b_tensor.shape
        
        assert N == N_b, f"矩阵维度不匹配: {N} != {N_b}"
        
        # 初始化输出矩阵 C (在 GPU 上分配)
        c_tensor = torch.zeros((M, K), device='cuda', dtype=torch.float32)
        
        # 确保输入是连续内存且在 GPU 上
        if not a_tensor.is_contiguous(): a_tensor = a_tensor.contiguous()
        if not b_tensor.is_contiguous(): b_tensor = b_tensor.contiguous()
        
        # 3. 调用 CUDA 函数
        # 注意：必须传入 data_ptr()，这是物理显存地址
        lib.solve(
            ctypes.c_void_p(a_tensor.data_ptr()),
            ctypes.c_void_p(b_tensor.data_ptr()),
            ctypes.c_void_p(c_tensor.data_ptr()),
            ctypes.c_int(M),
            ctypes.c_int(N),
            ctypes.c_int(K)
        )
        
        return c_tensor

    # 1. 运行你的 CUDA Kernel
    C_custom = cuda_matmul(A, B)
    print(f"{v}")
    # 3. 验证结果
    # 允许一点浮点误差
    if torch.allclose(C_custom, C_torch, atol=1e-3):
        print(f"✅ 测试通过！结果正确。")
        %timeit cuda_matmul(A, B); torch.cuda.synchronize()
    else:
        print("❌ 测试失败。结果不一致。")
        print("最大误差:", (C_custom - C_torch).abs().max().item())
    print()

正在进行矩阵乘法测试: [1024x1024] * [1024x1024]
Pytorch: 
592 µs ± 2.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

matmul_v5.so
✅ 测试通过！结果正确。
663 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



In [ ]:
%%writefile sgemm_kernel.cu
// 第6个版本
#include <cuda_runtime.h>
#include <stdio.h>

// -------------------------------------------------------------------------
// 参数定义 (针对 T4 优化)
// -------------------------------------------------------------------------
const int BM = 128;
const int BN = 128;
const int BK = 8;
const int TM = 8;
const int TN = 8;

__global__ __launch_bounds__(256)
void sgemm_optimized_t4_kernel(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    int tid = threadIdx.x;
    int ty = tid / 16;
    int tx = tid % 16;
    int by = blockIdx.y;
    int bx = blockIdx.x;
    
    __shared__ float As[BK][BM]; 
    __shared__ float Bs[BK][BN];

    float accum[TM][TN] = {0.0f};
    float rag[TM];
    float rbg[TN];

    const float* A_ptr = A + by * BM * K;
    const float* B_ptr = B + bx * BN;
    // C 指针后面计算

    int load_a_row = tid / 2; 
    int load_a_col = (tid % 2) * 4;
    int load_b_row = tid / 32;
    int load_b_col = (tid % 32) * 4;

    for (int k_step = 0; k_step < K; k_step += BK) {
        // Load A (Transposed)
        if (by * BM + load_a_row < M && k_step + load_a_col < K) {
            float4 tmp = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + k_step + load_a_col])[0];
            As[load_a_col + 0][load_a_row] = tmp.x;
            As[load_a_col + 1][load_a_row] = tmp.y;
            As[load_a_col + 2][load_a_row] = tmp.z;
            As[load_a_col + 3][load_a_row] = tmp.w;
        } else {
             As[load_a_col + 0][load_a_row] = 0.0f;
             As[load_a_col + 1][load_a_row] = 0.0f;
             As[load_a_col + 2][load_a_row] = 0.0f;
             As[load_a_col + 3][load_a_row] = 0.0f;
        }

        // Load B
        if (k_step + load_b_row < K && bx * BN + load_b_col < N) {
            float4 tmp = reinterpret_cast<const float4*>(&B_ptr[(k_step + load_b_row) * N + load_b_col])[0];
            reinterpret_cast<float4*>(&Bs[load_b_row][load_b_col])[0] = tmp;
        } else {
             Bs[load_b_row][load_b_col + 0] = 0.0f;
             Bs[load_b_row][load_b_col + 1] = 0.0f;
             Bs[load_b_row][load_b_col + 2] = 0.0f;
             Bs[load_b_row][load_b_col + 3] = 0.0f;
        }

        __syncthreads();

        #pragma unroll
        for (int k = 0; k < BK; ++k) {
            float4 tmpA0 = reinterpret_cast<const float4*>(&As[k][ty * TM])[0];
            float4 tmpA1 = reinterpret_cast<const float4*>(&As[k][ty * TM + 4])[0];
            rag[0] = tmpA0.x; rag[1] = tmpA0.y; rag[2] = tmpA0.z; rag[3] = tmpA0.w;
            rag[4] = tmpA1.x; rag[5] = tmpA1.y; rag[6] = tmpA1.z; rag[7] = tmpA1.w;

            float4 tmpB0 = reinterpret_cast<const float4*>(&Bs[k][tx * TN])[0];
            float4 tmpB1 = reinterpret_cast<const float4*>(&Bs[k][tx * TN + 4])[0];
            rbg[0] = tmpB0.x; rbg[1] = tmpB0.y; rbg[2] = tmpB0.z; rbg[3] = tmpB0.w;
            rbg[4] = tmpB1.x; rbg[5] = tmpB1.y; rbg[6] = tmpB1.z; rbg[7] = tmpB1.w;

            #pragma unroll
            for (int r = 0; r < TM; ++r) {
                #pragma unroll
                for (int c = 0; c < TN; ++c) {
                    accum[r][c] += rag[r] * rbg[c];
                }
            }
        }
        __syncthreads();
    }

    // Write back
    int global_row_start = by * BM + ty * TM;
    int global_col_start = bx * BN + tx * TN;

    #pragma unroll
    for (int r = 0; r < TM; ++r) {
        int global_r = global_row_start + r;
        if (global_r < M) {
            int global_c = global_col_start;
            if (global_c + 7 < N) {
                float4 tmp0, tmp1;
                tmp0.x = accum[r][0]; tmp0.y = accum[r][1]; tmp0.z = accum[r][2]; tmp0.w = accum[r][3];
                tmp1.x = accum[r][4]; tmp1.y = accum[r][5]; tmp1.z = accum[r][6]; tmp1.w = accum[r][7];
                reinterpret_cast<float4*>(&C[global_r * N + global_c])[0] = tmp0;
                reinterpret_cast<float4*>(&C[global_r * N + global_c + 4])[0] = tmp1;
            } else {
                for (int c = 0; c < TN; ++c) {
                    if (global_c + c < N) {
                        C[global_r * N + global_c + c] = accum[r][c];
                    }
                }
            }
        }
    }
}

// C++ 调用接口
void launch_sgemm_optimized(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 block(256);
    dim3 grid((N + BN - 1) / BN, (M + BM - 1) / BM);
    sgemm_optimized_t4_kernel<<<grid, block>>>(A, B, C, M, N, K);
}

Writing sgemm_kernel.cu


In [ ]:
%%writefile segemm_kernel.cu
// 第7个版本
#include <cuda_runtime.h>
#include <stdio.h>

// ==========================================
// T4 优化参数 + 双重缓冲策略
// ==========================================
const int BM = 128;
const int BN = 128;
const int BK = 8;
const int TM = 8;
const int TN = 8;

__global__ __launch_bounds__(256)
void sgemm_double_buffer_t4(
    const float* __restrict__ A, 
    const float* __restrict__ B, 
    float* __restrict__ C, 
    int M, int N, int K) 
{
    int tid = threadIdx.x;
    int ty = tid / 16;
    int tx = tid % 16;
    int by = blockIdx.y;
    int bx = blockIdx.x;
    
    // =========================================================
    // 关键改变 1: 双重缓冲 Shared Memory
    // 使用 [2] 个 buffer，write_stage 用于写，read_stage 用于算
    // =========================================================
    __shared__ float As[2][BK][BM]; 
    __shared__ float Bs[2][BK][BN];

    float accum[TM][TN] = {0.0f};
    
    // 寄存器缓存，用于计算
    float rag[TM];
    float rbg[TN];

    // 寄存器缓存，用于 Global Memory 预取 (Prefetch)
    // 每个线程搬运 1 个 float4 的 A 和 1 个 float4 的 B
    float4 load_a_reg; 
    float4 load_b_reg;

    const float* A_ptr = A + by * BM * K;
    const float* B_ptr = B + bx * BN;

    // 加载索引计算
    int load_a_row = tid / 2; 
    int load_a_col = (tid % 2) * 4;
    int load_b_row = tid / 32;
    int load_b_col = (tid % 32) * 4;

    // =========================================================
    // Prologue (序幕): 加载第一个 Tile 到 Buffer 0
    // =========================================================
    {
        int k_start = 0;
        // Load A
        if (by * BM + load_a_row < M && k_start + load_a_col < K) {
            load_a_reg = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + k_start + load_a_col])[0];
        } else {
            load_a_reg = {0.0f, 0.0f, 0.0f, 0.0f};
        }
        // Load B
        if (k_start + load_b_row < K && bx * BN + load_b_col < N) {
            load_b_reg = reinterpret_cast<const float4*>(&B_ptr[(k_start + load_b_row) * N + load_b_col])[0];
        } else {
            load_b_reg = {0.0f, 0.0f, 0.0f, 0.0f};
        }

        // 写入 SMEM Buffer 0
        // A 转置写入
        As[0][load_a_col + 0][load_a_row] = load_a_reg.x;
        As[0][load_a_col + 1][load_a_row] = load_a_reg.y;
        As[0][load_a_col + 2][load_a_row] = load_a_reg.z;
        As[0][load_a_col + 3][load_a_row] = load_a_reg.w;
        
        // B 直接写入
        reinterpret_cast<float4*>(&Bs[0][load_b_row][load_b_col])[0] = load_b_reg;
    }

    __syncthreads();

    // =========================================================
    // Main Loop
    // =========================================================
    int write_stage_idx = 1; // 下一轮写入的位置
    int read_stage_idx = 0;  // 当前计算读取的位置

    // 注意：循环从 k=0 开始算，但在 k 时我们要预加载 k+BK 的数据
    for (int k = 0; k < K; k += BK) {
        
        // -----------------------------------------------------
        // 1. Prefetch Next Tile to Registers (Global -> Register)
        // 这里的关键是：当我们发起 Global Load 指令后，GPU 不会阻塞，
        // 而是会继续向下执行计算指令 (Math)，从而隐藏内存延迟。
        // -----------------------------------------------------
        int next_k = k + BK;
        if (next_k < K) {
            // Load A to Reg
            if (by * BM + load_a_row < M && next_k + load_a_col < K) {
                load_a_reg = reinterpret_cast<const float4*>(&A_ptr[load_a_row * K + next_k + load_a_col])[0];
            } else {
                load_a_reg = {0.0f, 0.0f, 0.0f, 0.0f};
            }
            // Load B to Reg
            if (next_k + load_b_row < K && bx * BN + load_b_col < N) {
                load_b_reg = reinterpret_cast<const float4*>(&B_ptr[(next_k + load_b_row) * N + load_b_col])[0];
            } else {
                load_b_reg = {0.0f, 0.0f, 0.0f, 0.0f};
            }
        }

        // -----------------------------------------------------
        // 2. Compute Current Tile (Register <-> SMEM)
        // 使用 read_stage_idx
        // -----------------------------------------------------
        #pragma unroll
        for (int i = 0; i < BK; ++i) {
            // Load A from SMEM to Reg
            float4 tmpA0 = reinterpret_cast<const float4*>(&As[read_stage_idx][i][ty * TM])[0];
            float4 tmpA1 = reinterpret_cast<const float4*>(&As[read_stage_idx][i][ty * TM + 4])[0];
            rag[0] = tmpA0.x; rag[1] = tmpA0.y; rag[2] = tmpA0.z; rag[3] = tmpA0.w;
            rag[4] = tmpA1.x; rag[5] = tmpA1.y; rag[6] = tmpA1.z; rag[7] = tmpA1.w;

            // Load B from SMEM to Reg
            float4 tmpB0 = reinterpret_cast<const float4*>(&Bs[read_stage_idx][i][tx * TN])[0];
            float4 tmpB1 = reinterpret_cast<const float4*>(&Bs[read_stage_idx][i][tx * TN + 4])[0];
            rbg[0] = tmpB0.x; rbg[1] = tmpB0.y; rbg[2] = tmpB0.z; rbg[3] = tmpB0.w;
            rbg[4] = tmpB1.x; rbg[5] = tmpB1.y; rbg[6] = tmpB1.z; rbg[7] = tmpB1.w;

            // Compute
            #pragma unroll
            for (int r = 0; r < TM; ++r) {
                #pragma unroll
                for (int c = 0; c < TN; ++c) {
                    accum[r][c] += rag[r] * rbg[c];
                }
            }
        }

        // -----------------------------------------------------
        // 3. Store Prefetched Data to SMEM (Register -> SMEM)
        // 此时计算已经完成，我们等待所有线程都算完了当前块
        // -----------------------------------------------------
        __syncthreads(); // 确保 read_stage 的数据大家都不用了

        if (next_k < K) {
            // 将寄存器里的下一块数据写入 write_stage 的 SMEM
            // A Transposed
            As[write_stage_idx][load_a_col + 0][load_a_row] = load_a_reg.x;
            As[write_stage_idx][load_a_col + 1][load_a_row] = load_a_reg.y;
            As[write_stage_idx][load_a_col + 2][load_a_row] = load_a_reg.z;
            As[write_stage_idx][load_a_col + 3][load_a_row] = load_a_reg.w;

            // B Direct
            reinterpret_cast<float4*>(&Bs[write_stage_idx][load_b_row][load_b_col])[0] = load_b_reg;
        }

        // 翻转 buffer 索引
        // write_stage: 1 -> 0 -> 1
        // read_stage:  0 -> 1 -> 0
        write_stage_idx ^= 1;
        read_stage_idx ^= 1;

        __syncthreads(); // 确保 write_stage 的数据已经写好，可以作为下一轮的 read_stage
    }

    // =========================================================
    // Write Back
    // =========================================================
    int global_row_start = by * BM + ty * TM;
    int global_col_start = bx * BN + tx * TN;

    #pragma unroll
    for (int r = 0; r < TM; ++r) {
        int global_r = global_row_start + r;
        if (global_r < M) {
            int global_c = global_col_start;
            if (global_c + 7 < N) {
                float4 tmp0, tmp1;
                tmp0.x = accum[r][0]; tmp0.y = accum[r][1]; tmp0.z = accum[r][2]; tmp0.w = accum[r][3];
                tmp1.x = accum[r][4]; tmp1.y = accum[r][5]; tmp1.z = accum[r][6]; tmp1.w = accum[r][7];
                reinterpret_cast<float4*>(&C[global_r * N + global_c])[0] = tmp0;
                reinterpret_cast<float4*>(&C[global_r * N + global_c + 4])[0] = tmp1;
            } else {
                for (int c = 0; c < TN; ++c) {
                    if (global_c + c < N) {
                        C[global_r * N + global_c + c] = accum[r][c];
                    }
                }
            }
        }
    }
}

// 宿主调用
void launch_sgemm_optimized(const float* A, const float* B, float* C, int M, int N, int K) {
    dim3 block(256);
    dim3 grid((N + BN - 1) / BN, (M + BM - 1) / BM);
    sgemm_double_buffer_t4<<<grid, block>>>(A, B, C, M, N, K);
}

Writing segemm_kernel.cu


In [2]:
%%writefile sgemm_binding.cpp
#include <torch/extension.h>
#include <vector>

// 声明 CUDA 函数
void launch_sgemm_optimized(const float* A, const float* B, float* C, int M, int N, int K);

// 检查 Tensor 是否符合要求
#define CHECK_CUDA(x) TORCH_CHECK(x.is_cuda(), #x " must be a CUDA tensor")
#define CHECK_CONTIGUOUS(x) TORCH_CHECK(x.is_contiguous(), #x " must be contiguous")
#define CHECK_INPUT(x) CHECK_CUDA(x); CHECK_CONTIGUOUS(x)

torch::Tensor sgemm_forward(torch::Tensor a, torch::Tensor b) {
    // 1. 检查输入
    CHECK_INPUT(a);
    CHECK_INPUT(b);

    // 2. 获取维度
    // 假设 a 是 (M, K), b 是 (K, N)
    int M = a.size(0);
    int K = a.size(1);
    int K_b = b.size(0);
    int N = b.size(1);

    TORCH_CHECK(K == K_b, "Shape mismatch: a.shape[1] must equal b.shape[0]");

    // 3. 申请输出内存
    // 创建一个 (M, N) 的 float32 tensor
    auto c = torch::zeros({M, N}, a.options());

    // 4. 调用 CUDA Kernel
    launch_sgemm_optimized(
        a.data_ptr<float>(),
        b.data_ptr<float>(),
        c.data_ptr<float>(),
        M, N, K
    );

    return c;
}

// 5. 绑定到 Python 模块
PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("matmul", &sgemm_forward, "Optimized SGEMM for T4 (CUDA)");
}

Overwriting sgemm_binding.cpp


In [3]:
%%writefile setup.py
from setuptools import setup
from torch.utils.cpp_extension import BuildExtension, CUDAExtension

setup(
    name='mysgemm',
    ext_modules=[
        CUDAExtension(
            name='mysgemm_cuda', # 编译后的包名
            sources=['sgemm_binding.cpp', 'sgemm_kernel.cu'],
            extra_compile_args={
                'cxx': ['-O3'],
                'nvcc': [
                    '-O3',
                    '-gencode=arch=compute_75,code=sm_75', # 针对 T4 优化
                    '--use_fast_math'
                ]
            }
        )
    ],
    cmdclass={
        'build_ext': BuildExtension
    }
)

Overwriting setup.py


In [4]:
!python setup.py build_ext --inplace

running build_ext
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:448: UserWarning: The detected CUDA version (12.5) has a minor version mismatch with the version that was used to compile PyTorch (12.4). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:458: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.5
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
building 'mysgemm_cuda' extension
Emitting ninja build file /kaggle/working/build/temp.linux-x86_64-cpython-311/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/1] c++ -MMD -MF /kaggle/working/build/temp.linux-x86_64-cpython-311/sgemm_binding.o.d -Wsign-compare -DNDEBUG -g -

In [5]:
import torch
import mysgemm_cuda # 导入编译好的模块
import time

# 确保在 CUDA 上
device = torch.device("cuda")

# 矩阵大小 (尽量使用 128 的倍数以获得最佳性能，虽然代码兼容非倍数)
M, N, K = 4096, 4096, 4096

print(f"Testing SGEMM {M}x{N}x{K} on T4...")

# 初始化数据
# 注意：我们的 C++ 代码要求输入是 contiguous 的
a = torch.randn(M, K, device=device, dtype=torch.float32).contiguous()
b = torch.randn(K, N, device=device, dtype=torch.float32).contiguous()

# 预热
for _ in range(10):
    c_custom = mysgemm_cuda.matmul(a, b)

# 计时：自定义 CUDA
torch.cuda.synchronize()
start = time.time()
for _ in range(100):
    c_custom = mysgemm_cuda.matmul(a, b)
torch.cuda.synchronize()
custom_time = (time.time() - start) / 100
print(f"Custom Kernel Time: {custom_time*1000:.3f} ms")

# 计时：PyTorch (cuBLAS)
torch.cuda.synchronize()
start = time.time()
for _ in range(100):
    c_torch = torch.matmul(a, b)
torch.cuda.synchronize()
torch_time = (time.time() - start) / 100
print(f"PyTorch (cuBLAS) Time: {torch_time*1000:.3f} ms")

# 验证正确性
# 允许一点浮点误差
diff = (c_custom - c_torch).abs().max()
print(f"Max absolute difference: {diff.item()}")

if diff < 1e-2:
    print("✅ Correctness check passed!")
else:
    print("❌ Correctness check failed!")

Testing SGEMM 4096x4096x4096 on T4...
Custom Kernel Time: 38.811 ms
PyTorch (cuBLAS) Time: 35.835 ms
Max absolute difference: 0.0
✅ Correctness check passed!
